# Requirements

In [1]:
!pip install tqdm pandas requests

In [6]:
import requests
import pandas as pd
from requests.exceptions import HTTPError, ConnectionError, Timeout
from io import StringIO
import time
from tqdm import tqdm
from datetime import datetime
import json

In [7]:
# List of URLs of the CSV files for different indices
urls = { 
    "NIFTY 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv",
    "NIFTY NEXT 50": "https://www.niftyindices.com/IndexConstituent/ind_niftynext50list.csv",
    "NIFTY 100": "https://www.niftyindices.com/IndexConstituent/ind_nifty100list.csv",
    "NIFTY 200": "https://www.niftyindices.com/IndexConstituent/ind_nifty200list.csv",
    "NIFTY TOTAL MKT": "https://www.niftyindices.com/IndexConstituent/ind_niftytotalmarket_list.csv",
    "NIFTY 500": "https://www.niftyindices.com/IndexConstituent/ind_nifty500list.csv",
    "NIFTY500 MULTICAP": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Multicap502525_list.csv",
    "NIFTY MIDCAP150": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150list.csv",
    "NIFTY MIDCAP 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap50list.csv",
    "NIFTY MID SELECT": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcapselect_list.csv",
    "NIFTY MIDCAP 100": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap100list.csv",
    "NIFTY SMLCAP 250": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap250list.csv",
    "NIFTY SMLCAP 50": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap50list.csv",
    "NIFTY SMLCAP 100": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap100list.csv",
    "NIFTY MICROCAP250": "https://www.niftyindices.com/IndexConstituent/ind_niftymicrocap250_list.csv",
    "NIFTY LARGEMID250": "https://www.niftyindices.com/IndexConstituent/ind_niftylargemidcap250list.csv",
    "NIFTY MIDSML 400": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallcap400list.csv",
    "NIFTY500 LARGEMIDSMALL EQUAL-CAP WEIGHTED":"https://www.niftyindices.com/IndexConstituent/ind_nifty500LargeMidSmallEqualCapWeighted_list.csv",
    "NIFTY AUTO": "https://www.niftyindices.com/IndexConstituent/ind_niftyautolist.csv",
    "NIFTY BANK": "https://www.niftyindices.com/IndexConstituent/ind_niftybanklist.csv",
    "NIFTY FIN SERVICE": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancelist.csv",
    "NIFTY FINSRV25 50": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservices25-50list.csv",
    "NIFTY FINANCIAL SERVICES EX BANK": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservicesexbank_list.csv",
    "NIFTY FMCG": "https://www.niftyindices.com/IndexConstituent/ind_niftyfmcglist.csv",
    "NIFTY HEALTHCARE": "https://www.niftyindices.com/IndexConstituent/ind_niftyhealthcarelist.csv",
    "NIFTY IT": "https://www.niftyindices.com/IndexConstituent/ind_niftyitlist.csv",
    "NIFTY MEDIA": "https://www.niftyindices.com/IndexConstituent/ind_niftymedialist.csv",
    "NIFTY METAL": "https://www.niftyindices.com/IndexConstituent/ind_niftymetallist.csv",
    "NIFTY PHARMA": "https://www.niftyindices.com/IndexConstituent/ind_niftypharmalist.csv",
    "NIFTY PVT BANK": "https://www.niftyindices.com/IndexConstituent/ind_nifty_privatebanklist.csv",
    "NIFTY PSU BANK": "https://www.niftyindices.com/IndexConstituent/ind_niftypsubanklist.csv",
    "NIFTY REALTY": "https://www.niftyindices.com/IndexConstituent/ind_niftyrealtylist.csv",
    "NIFTY CONSR DURBL": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumerdurableslist.csv",
    "NIFTY OIL AND GAS": "https://www.niftyindices.com/IndexConstituent/ind_niftyoilgaslist.csv",
    "NIFTY MIDSMALL FINANCIAL SERVICES": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallfinancailservice_list.csv",
    "NIFTY MIDSMALL HEALTHCARE": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallhealthcare_list.csv",
    "NIFTY MIDSMALL IT & TELECOM": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallitAndtelecom_list.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - ADITYA BIRLA GROUP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_adityabirlalist.csv",
    "NIFTY COMMODITIES": "https://www.niftyindices.com/IndexConstituent/ind_niftycommoditieslist.csv",
    "NIFTY CORE HOUSING": "https://www.niftyindices.com/IndexConstituent/ind_niftyCoreHousing_list.csv",
    "NIFTY CPSE": "https://www.niftyindices.com/IndexConstituent/ind_niftycpselist.csv",
    "NIFTY ENERGY": "https://www.niftyindices.com/IndexConstituent/ind_niftyenergylist.csv",
    "NIFTY HOUSING": "https://www.niftyindices.com/IndexConstituent/ind_niftyhousing_list.csv",
    "NIFTY CONSUMPTION": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumptionlist.csv",
    "NIFTY INDIA DEFENCE": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadefence_list.csv",
    "NIFTY IND DIGITAL": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadigital_list.csv",
    "NIFTY INDIA MFG": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiamanufacturing_list.csv",
    "NIFTY INFRA": "https://www.niftyindices.com/IndexConstituent/ind_niftyinfralist.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - MAHINDRA GROUP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_mahindralist.csv",
    "NIFTY MID LIQ 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty_Midcap_Liquid15.csv",
    "NIFTY MIDSMALL INDIA CONSUMPTION": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallindiaconsumption_list.csv",
    "NIFTY MNC": "https://www.niftyindices.com/IndexConstituent/ind_niftymnclist.csv",
    "NIFTY MOBILITY": "https://www.niftyindices.com/IndexConstituent/ind_niftymobility_list.csv",
    "NIFTY PSE": "https://www.niftyindices.com/IndexConstituent/ind_niftypselist.csv",
    "NIFTY REITS & INVITS": "https://www.niftyindices.com/IndexConstituent/ind_niftyREITs_InvITs_list.csv",
    "NIFTY NON-CYCLICAL CONSUMER": "https://www.niftyindices.com/IndexConstituent/ind_niftynon-cyclicalconsumer_list.csv",
    "NIFTY SERV SECTOR": "https://www.niftyindices.com/IndexConstituent/ind_niftyservicelist.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - TATA GROUP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tatalist.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - TATA GROUP 25% CAP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tata25caplist.csv",
    "NIFTY TRANSPORTATION & LOGISTICS": "https://www.niftyindices.com/IndexConstituent/ind_niftytransportationandlogistics%20_list.csv",
    "NIFTY EV & NEW AGE AUTOMOTIVE":"https://www.niftyindices.com/Index_Statistics/ind_niftyEv_NewAgeAutomotive_list.csv",
    "NIFTY INDIA TOURISM":"https://www.niftyindices.com/IndexConstituent/ind_niftyindiatourism_list.csv",
    "NIFTY100 LIQUID 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty100_Liquid15.csv",
    "NIFTY500 MULTICAP INDIA MANUFACTURING 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapIndiaManufacturing503020_list.csv",
    "NIFTY500 MULTICAP INFRASTRUCTURE 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapInfrastructure503020_list.csv",
    "NIFTY SME EMERGE": "https://www.niftyindices.com/IndexConstituent/ind_niftysmelist.csv",
    "NIFTY100 EQL WGT": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "NIFTY100 LOWVOL30": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "NIFTY200 MOMENTM30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Momentum30_list.csv",
    "NIFTY200 ALPHA 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200alpha30_list.csv",
    "NIFTY100 ALPHA 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Alpha30list.csv",
    "NIFTY ALPHA 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty_Alpha_Index.csv",
    "NIFTY ALPHA LOW VOLATILITY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_lowvol30list.csv",
    "NIFTY ALPHA QUALITY LOW VOLATILITY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_lowvol30list.csv",
    "NIFTY DIV OPPS 50": "https://www.niftyindices.com/IndexConstituent/ind_niftydivopp50list.csv",
    "NIFTY HIGH BETA 50": "https://www.niftyindices.com/IndexConstituent/nifty_High_Beta50_Index.csv",
    "NIFTY LOW VOLATILITY 50": "https://www.niftyindices.com/IndexConstituent/nifty_low_Volatility50_Index.csv",
    "NIFTY100 QUALTY30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Quality30list.csv",
    "NIFTY MIDCAP150 MOMENTUM 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150momentum50_list.csv",
    "NIFTY MIDCAP150 QUALITY 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150quality50list.csv",
    "NIFTY SMALLCAP250 QUALITY 50 INDEX": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250_Quality50_list.csv",
    "NIFTY MIDSMALLCAP400 MOMENTUM QUALITY 100": "https://www.niftyindices.com/IndexConstituent/ind_niftyMidSmallcap400MomentumQuality100_list.csv",
    "NIFTY SMALLCAP250 MOMENTUM QUALITY 100": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250MomentumQuality100_list.csv",
    "NIFTY QUALITY LOW VOLATILITY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_quality_lowvol30list.csv",
    "NIFTY50 EQL WGT": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "NIFTY50 VALUE 20": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "NIFTY500 VALUE 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Value50_list.csv",
    "NIFTY200 QUALTY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Quality30_list.csv",
    "NIFTY GROWSECT 15":"https://www.niftyindices.com/IndexConstituent/ind_NiftyGrowth_Sectors15_Index.csv",
    "NIFTY500 EQUAL WEIGHT":"https://www.niftyindices.com/IndexConstituent/ind_nifty500EqualWeight_list.csv",
    "NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_value_lowvol30list.csv",
    "NIFTY500 MOMENTUM 50":"https://www.niftyindices.com/IndexConstituent/ind_nifty500Momentum50_list.csv",
    "NIFTY200 VALUE 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty200Value30_list.csv",
    "NIFTY TOP 10 EQUAL WEIGHT":"https://www.niftyindices.com/IndexConstituent/ind_niftytop10EqualWeight_list.csv"
}

# Headers to mimic a browser request
headers = {
    "User-Agent": "PostmanRuntime/7.39.0"
}

def fetch_csv_data(url, retries=3, backoff_factor=1):
    """
    Fetches CSV data from the given URL with specified retries and backoff factor.
    """
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            return response.content.decode('utf-8')
        except (HTTPError, ConnectionError, Timeout) as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(backoff_factor * (2 ** attempt))
            else:
                raise

def process_csv_data(url):
    """
    Processes CSV data from the given URL and returns a DataFrame with an 'Index Name' column.
    """
    try:
        csv_data = fetch_csv_data(url)
        
        if csv_data:
            df = pd.read_csv(StringIO(csv_data))
            df['Index Name'] = url.split('/')[-1].replace('.csv', '')  # Add column for index name
            return df
        else:
            print(f"No data returned from {url}")
    except Exception as e:
        print(f"Error fetching or processing the CSV file from {url}: {e}")

def get_current_quarter_year():
    """
    Gets the current quarter and year.
    """
    current_date = datetime.now()
    current_year = current_date.year
    current_quarter = (current_date.month - 1) // 3 + 1
    return current_quarter, current_year

def save_nifty_total_market_csv():
    """
    Fetches the Nifty Total Market CSV data and saves it to a file.
    """
    url = "https://nsearchives.nseindia.com/content/indices/ind_niftytotalmarket_list.csv"
    try:
        csv_data = fetch_csv_data(url)
        if csv_data:
            current_quarter, current_year = get_current_quarter_year()
            filename = f'nifty-tmi-Q{current_quarter}-{current_year}.csv'
            with open(filename, 'w') as file:
                file.write(csv_data)
            print(f"Nifty Total Market data saved to {filename}")
        else:
            print("No data returned from the Nifty Total Market URL.")
    except Exception as e:
        print(f"Error fetching or saving the Nifty Total Market CSV data: {e}")

def indices_list_update():
    """
    Fetches and saves all indices data into a single CSV file and a JSON file.
    """
    indices_data = {}
    all_data = []

    for index_name, url in tqdm(urls.items(), desc="Processing Indices"):
        try:
            df = process_csv_data(url)
            if df is not None:
                all_data.append(df)
                indices_data[index_name] = {"index name": index_name, "stock list": df['Symbol'].tolist()}
        except Exception as e:
            print(f"Error processing data from {url}: {e}")

    # Save the indices data to a JSON file
    current_quarter, current_year = get_current_quarter_year()
    json_filename = f'indices-data-Q{current_quarter}-{current_year}.json'
    with open(json_filename, 'w') as json_file:
        json.dump(indices_data, json_file, indent=4)
    print(f"Data saved to {json_filename}")

    # Save the combined DataFrame to a CSV file
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        csv_filename = 'indices_csv.csv'
        combined_df.to_csv(csv_filename, index=False)
        print(f"All indices data saved to {csv_filename}")
    else:
        print("No data was collected from the indices.")

def is_first_day_of_quarter():
    """
    Checks if today is the first day of the quarter.
    """
    today = datetime.now()
    today=datetime(2024,7,1) # just for the testing purpose
    first_day_of_quarter = datetime(today.year, ((today.month - 1) // 3) * 3 + 1, 1)
    return today.date() == first_day_of_quarter.date()

if __name__ == "__main__":
    if is_first_day_of_quarter():
        save_nifty_total_market_csv()
        indices_list_update()
    else:
        print("Not executing code for TMI list and INDICES list. Exiting the script.")


Nifty Total Market data saved to nifty-tmi-Q3-2024.csv


Processing Indices: 100%|██████████| 94/94 [00:55<00:00,  1.69it/s]

Data saved to indices-data-Q3-2024.json
All indices data saved to indices_csv.csv


# url list

In [ ]:
# List of URLs of the CSV files for different indices
urls = { 
    "NIFTY 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv",
    "NIFTY NEXT 50": "https://www.niftyindices.com/IndexConstituent/ind_niftynext50list.csv",
    "NIFTY 100": "https://www.niftyindices.com/IndexConstituent/ind_nifty100list.csv",
    "NIFTY 200": "https://www.niftyindices.com/IndexConstituent/ind_nifty200list.csv",
    "NIFTY TOTAL MKT": "https://www.niftyindices.com/IndexConstituent/ind_niftytotalmarket_list.csv",
    "NIFTY 500": "https://www.niftyindices.com/IndexConstituent/ind_nifty500list.csv",
    "NIFTY500 MULTICAP": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Multicap502525_list.csv",
    "NIFTY MIDCAP150": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150list.csv",
    "NIFTY MIDCAP 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap50list.csv",
    "NIFTY MID SELECT": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcapselect_list.csv",
    "NIFTY MIDCAP 100": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap100list.csv",
    "NIFTY SMLCAP 250": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap250list.csv",
    "NIFTY SMLCAP 50": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap50list.csv",
    "NIFTY SMLCAP 100": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap100list.csv",
    "NIFTY MICROCAP250": "https://www.niftyindices.com/IndexConstituent/ind_niftymicrocap250_list.csv",
    "NIFTY LARGEMID250": "https://www.niftyindices.com/IndexConstituent/ind_niftylargemidcap250list.csv",
    "NIFTY MIDSML 400": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallcap400list.csv",
    "NIFTY500 LARGEMIDSMALL EQUAL-CAP WEIGHTED":"https://www.niftyindices.com/IndexConstituent/ind_nifty500LargeMidSmallEqualCapWeighted_list.csv",
    "NIFTY AUTO": "https://www.niftyindices.com/IndexConstituent/ind_niftyautolist.csv",
    "NIFTY BANK": "https://www.niftyindices.com/IndexConstituent/ind_niftybanklist.csv",
    "NIFTY FIN SERVICE": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancelist.csv",
    "NIFTY FINSRV25 50": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservices25-50list.csv",
    "NIFTY FINANCIAL SERVICES EX BANK": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservicesexbank_list.csv",
    "NIFTY FMCG": "https://www.niftyindices.com/IndexConstituent/ind_niftyfmcglist.csv",
    "NIFTY HEALTHCARE": "https://www.niftyindices.com/IndexConstituent/ind_niftyhealthcarelist.csv",
    "NIFTY IT": "https://www.niftyindices.com/IndexConstituent/ind_niftyitlist.csv",
    "NIFTY MEDIA": "https://www.niftyindices.com/IndexConstituent/ind_niftymedialist.csv",
    "NIFTY METAL": "https://www.niftyindices.com/IndexConstituent/ind_niftymetallist.csv",
    "NIFTY PHARMA": "https://www.niftyindices.com/IndexConstituent/ind_niftypharmalist.csv",
    "NIFTY PVT BANK": "https://www.niftyindices.com/IndexConstituent/ind_nifty_privatebanklist.csv",
    "NIFTY PSU BANK": "https://www.niftyindices.com/IndexConstituent/ind_niftypsubanklist.csv",
    "NIFTY REALTY": "https://www.niftyindices.com/IndexConstituent/ind_niftyrealtylist.csv",
    "NIFTY CONSR DURBL": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumerdurableslist.csv",
    "NIFTY OIL AND GAS": "https://www.niftyindices.com/IndexConstituent/ind_niftyoilgaslist.csv",
    "NIFTY MIDSMALL FINANCIAL SERVICES": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallfinancailservice_list.csv",
    "NIFTY MIDSMALL HEALTHCARE": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallhealthcare_list.csv",
    "NIFTY MIDSMALL IT & TELECOM": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallitAndtelecom_list.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - ADITYA BIRLA GROUP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_adityabirlalist.csv",
    "NIFTY COMMODITIES": "https://www.niftyindices.com/IndexConstituent/ind_niftycommoditieslist.csv",
    "NIFTY CORE HOUSING": "https://www.niftyindices.com/IndexConstituent/ind_niftyCoreHousing_list.csv",
    "NIFTY CPSE": "https://www.niftyindices.com/IndexConstituent/ind_niftycpselist.csv",
    "NIFTY ENERGY": "https://www.niftyindices.com/IndexConstituent/ind_niftyenergylist.csv",
    "NIFTY HOUSING": "https://www.niftyindices.com/IndexConstituent/ind_niftyhousing_list.csv",
    "NIFTY CONSUMPTION": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumptionlist.csv",
    "NIFTY INDIA DEFENCE": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadefence_list.csv",
    "NIFTY IND DIGITAL": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadigital_list.csv",
    "NIFTY INDIA MFG": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiamanufacturing_list.csv",
    "NIFTY INFRA": "https://www.niftyindices.com/IndexConstituent/ind_niftyinfralist.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - MAHINDRA GROUP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_mahindralist.csv",
    "NIFTY MID LIQ 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty_Midcap_Liquid15.csv",
    "NIFTY MIDSMALL INDIA CONSUMPTION": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallindiaconsumption_list.csv",
    "NIFTY MNC": "https://www.niftyindices.com/IndexConstituent/ind_niftymnclist.csv",
    "NIFTY MOBILITY": "https://www.niftyindices.com/IndexConstituent/ind_niftymobility_list.csv",
    "NIFTY PSE": "https://www.niftyindices.com/IndexConstituent/ind_niftypselist.csv",
    "NIFTY REITS & INVITS": "https://www.niftyindices.com/IndexConstituent/ind_niftyREITs_InvITs_list.csv",
    "NIFTY NON-CYCLICAL CONSUMER": "https://www.niftyindices.com/IndexConstituent/ind_niftynon-cyclicalconsumer_list.csv",
    "NIFTY SERV SECTOR": "https://www.niftyindices.com/IndexConstituent/ind_niftyservicelist.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - TATA GROUP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tatalist.csv",
    "NIFTY INDIA CORPORATE GROUP INDEX - TATA GROUP 25% CAP": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tata25caplist.csv",
    "NIFTY TRANSPORTATION & LOGISTICS": "https://www.niftyindices.com/IndexConstituent/ind_niftytransportationandlogistics%20_list.csv",
    "NIFTY EV & NEW AGE AUTOMOTIVE":"https://www.niftyindices.com/Index_Statistics/ind_niftyEv_NewAgeAutomotive_list.csv",
    "NIFTY INDIA TOURISM":"https://www.niftyindices.com/IndexConstituent/ind_niftyindiatourism_list.csv",
    "NIFTY100 LIQUID 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty100_Liquid15.csv",
    "NIFTY500 MULTICAP INDIA MANUFACTURING 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapIndiaManufacturing503020_list.csv",
    "NIFTY500 MULTICAP INFRASTRUCTURE 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapInfrastructure503020_list.csv",
    "NIFTY SME EMERGE": "https://www.niftyindices.com/IndexConstituent/ind_niftysmelist.csv",
    "NIFTY100 EQL WGT": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "NIFTY100 LOWVOL30": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "NIFTY200 MOMENTM30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Momentum30_list.csv",
    "NIFTY200 ALPHA 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200alpha30_list.csv",
    "NIFTY100 ALPHA 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Alpha30list.csv",
    "NIFTY ALPHA 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty_Alpha_Index.csv",
    "NIFTY ALPHA LOW VOLATILITY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_lowvol30list.csv",
    "NIFTY ALPHA QUALITY LOW VOLATILITY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_lowvol30list.csv",
    "NIFTY DIV OPPS 50": "https://www.niftyindices.com/IndexConstituent/ind_niftydivopp50list.csv",
    "NIFTY HIGH BETA 50": "https://www.niftyindices.com/IndexConstituent/nifty_High_Beta50_Index.csv",
    "NIFTY LOW VOLATILITY 50": "https://www.niftyindices.com/IndexConstituent/nifty_low_Volatility50_Index.csv",
    "NIFTY100 QUALTY30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Quality30list.csv",
    "NIFTY MIDCAP150 MOMENTUM 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150momentum50_list.csv",
    "NIFTY MIDCAP150 QUALITY 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150quality50list.csv",
    "NIFTY SMALLCAP250 QUALITY 50 INDEX": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250_Quality50_list.csv",
    "NIFTY MIDSMALLCAP400 MOMENTUM QUALITY 100": "https://www.niftyindices.com/IndexConstituent/ind_niftyMidSmallcap400MomentumQuality100_list.csv",
    "NIFTY SMALLCAP250 MOMENTUM QUALITY 100": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250MomentumQuality100_list.csv",
    "NIFTY QUALITY LOW VOLATILITY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_quality_lowvol30list.csv",
    "NIFTY50 EQL WGT": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "NIFTY50 VALUE 20": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "NIFTY500 VALUE 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Value50_list.csv",
    "NIFTY200 QUALTY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Quality30_list.csv",
    "NIFTY GROWSECT 15":"https://www.niftyindices.com/IndexConstituent/ind_NiftyGrowth_Sectors15_Index.csv",
    "NIFTY500 EQUAL WEIGHT":"https://www.niftyindices.com/IndexConstituent/ind_nifty500EqualWeight_list.csv",
    "NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_value_lowvol30list.csv",
    "NIFTY500 MOMENTUM 50":"https://www.niftyindices.com/IndexConstituent/ind_nifty500Momentum50_list.csv",
    "NIFTY200 VALUE 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty200Value30_list.csv",
    "NIFTY TOP 10 EQUAL WEIGHT":"https://www.niftyindices.com/IndexConstituent/ind_niftytop10EqualWeight_list.csv"
}

# long name

In [ ]:
urls = { 
    "Nifty 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv",
    "Nifty Next 50": "https://www.niftyindices.com/IndexConstituent/ind_niftynext50list.csv",
    "Nifty 100": "https://www.niftyindices.com/IndexConstituent/ind_nifty100list.csv",
    "Nifty 200": "https://www.niftyindices.com/IndexConstituent/ind_nifty200list.csv",
    "Nifty Total Market": "https://www.niftyindices.com/IndexConstituent/ind_niftytotalmarket_list.csv",
    "Nifty 500": "https://www.niftyindices.com/IndexConstituent/ind_nifty500list.csv",
    "Nifty500 Multicap 50:25:25": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Multicap502525_list.csv",
    "Nifty Midcap150": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150list.csv",
    "Nifty Midcap 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap50list.csv",
    "Nifty Midcap Select": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcapselect_list.csv",
    "Nifty Midcap 100": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap100list.csv",
    "Nifty Smallcap 250": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap250list.csv",
    "Nifty Smallcap 50": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap50list.csv",
    "Nifty Smallcap 100": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap100list.csv",
    "Nifty Microcap 250": "https://www.niftyindices.com/IndexConstituent/ind_niftymicrocap250_list.csv",
    "Nifty LargeMidcap 250": "https://www.niftyindices.com/IndexConstituent/ind_niftylargemidcap250list.csv",
    "Nifty MidSmallcap 400": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallcap400list.csv",
    "Nifty Auto": "https://www.niftyindices.com/IndexConstituent/ind_niftyautolist.csv",
    "Nifty Bank": "https://www.niftyindices.com/IndexConstituent/ind_niftybanklist.csv",
    "Nifty Financial Services": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancelist.csv",
    "Nifty Financial Services 25/50": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservices25-50list.csv",
    "Nifty Financial Services Ex-Bank": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservicesexbank_list.csv",
    "Nifty FMCG": "https://www.niftyindices.com/IndexConstituent/ind_niftyfmcglist.csv",
    "Nifty Healthcare": "https://www.niftyindices.com/IndexConstituent/ind_niftyhealthcarelist.csv",
    "Nifty IT": "https://www.niftyindices.com/IndexConstituent/ind_niftyitlist.csv",
    "Nifty Media": "https://www.niftyindices.com/IndexConstituent/ind_niftymedialist.csv",
    "Nifty Metal": "https://www.niftyindices.com/IndexConstituent/ind_niftymetallist.csv",
    "Nifty Pharma": "https://www.niftyindices.com/IndexConstituent/ind_niftypharmalist.csv",
    "Nifty Private Bank": "https://www.niftyindices.com/IndexConstituent/ind_nifty_privatebanklist.csv",
    "Nifty PSU Bank": "https://www.niftyindices.com/IndexConstituent/ind_niftypsubanklist.csv",
    "Nifty Realty": "https://www.niftyindices.com/IndexConstituent/ind_niftyrealtylist.csv",
    "Nifty Consumer Durables": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumerdurableslist.csv",
    "Nifty Oil and Gas": "https://www.niftyindices.com/IndexConstituent/ind_niftyoilgaslist.csv",
    "Nifty MidSmall Financial Services": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallfinancailservice_list.csv",
    "Nifty MidSmall Healthcare": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallhealthcare_list.csv",
    "Nifty MidSmall IT & Telecom": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallitAndtelecom_list.csv",
    "Nifty India Corporate Group Index - Aditya Birla Group": "https://www.niftyindices.com/IndexConstituent/ind_nifty_adityabirlalist.csv",
    "Nifty Commodities": "https://www.niftyindices.com/IndexConstituent/ind_niftycommoditieslist.csv",
    "Nifty Core Housing": "https://www.niftyindices.com/IndexConstituent/ind_niftyCoreHousing_list.csv",
    "Nifty CPSE": "https://www.niftyindices.com/IndexConstituent/ind_niftycpselist.csv",
    "Nifty Energy": "https://www.niftyindices.com/IndexConstituent/ind_niftyenergylist.csv",
    "Nifty Housing": "https://www.niftyindices.com/IndexConstituent/ind_niftyhousing_list.csv",
    "Nifty India Consumption": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumptionlist.csv",
    "Nifty India Defence": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadefence_list.csv",
    "Nifty India Digital": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadigital_list.csv",
    "Nifty India Manufacturing": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiamanufacturing_list.csv",
    "Nifty Infrastructure": "https://www.niftyindices.com/IndexConstituent/ind_niftyinfralist.csv",
    "Nifty India Corporate Group Index - Mahindra Group": "https://www.niftyindices.com/IndexConstituent/ind_nifty_mahindralist.csv",
    "Nifty Midcap Liquid 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty_Midcap_Liquid15.csv",
    "Nifty MidSmall India Consumption": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallindiaconsumption_list.csv",
    "Nifty MNC": "https://www.niftyindices.com/IndexConstituent/ind_niftymnclist.csv",
    "Nifty Mobility": "https://www.niftyindices.com/IndexConstituent/ind_niftymobility_list.csv",
    "Nifty PSE": "https://www.niftyindices.com/IndexConstituent/ind_niftypselist.csv",
    "Nifty REITs & InvITs": "https://www.niftyindices.com/IndexConstituent/ind_niftyREITs_InvITs_list.csv",
    "Nifty Non-Cyclical Consumer": "https://www.niftyindices.com/IndexConstituent/ind_niftynon-cyclicalconsumer_list.csv",
    "Nifty Services Sector": "https://www.niftyindices.com/IndexConstituent/ind_niftyservicelist.csv",
    "Nifty India Corporate Group Index - Tata Group": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tatalist.csv",
    "Nifty India Corporate Group Index - Tata Group 25% Cap": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tata25caplist.csv",
    "Nifty Transportation & Logistics": "https://www.niftyindices.com/IndexConstituent/ind_niftytransportationandlogistics%20_list.csv",
    "Nifty100 Liquid 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty100_Liquid15.csv",
    "Nifty500 Multicap India Manufacturing 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapIndiaManufacturing503020_list.csv",
    "Nifty500 Multicap Infrastructure 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapInfrastructure503020_list.csv",
    "Nifty SME Emerge": "https://www.niftyindices.com/IndexConstituent/ind_niftysmelist.csv",
    "Nifty100 Equal Weight": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "Nifty100 Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "Nifty200 Momentum 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Momentum30_list.csv",
    "Nifty200 Alpha 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200alpha30_list.csv",
    "Nifty100 Alpha 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Alpha30list.csv",
    "Nifty Alpha 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty_Alpha_Index.csv",
    "Nifty Alpha Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_lowvol30list.csv",
    "Nifty Alpha Quality Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_lowvol30list.csv",
    "Nifty Dividend Opportunities 50": "https://www.niftyindices.com/IndexConstituent/ind_niftydivopp50list.csv",
    "Nifty High Beta 50": "https://www.niftyindices.com/IndexConstituent/nifty_High_Beta50_Index.csv",
    "Nifty Low Volatility 50": "https://www.niftyindices.com/IndexConstituent/nifty_low_Volatility50_Index.csv",
    "Nifty100 Quality 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Quality30list.csv",
    "Nifty Midcap150 Momentum 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150momentum50_list.csv",
    "Nifty Midcap150 Quality 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150quality50list.csv",
    "Nifty Smallcap250 Quality 50": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250_Quality50_list.csv",
    "Nifty MidSmallcap400 Momentum Quality 100": "https://www.niftyindices.com/IndexConstituent/ind_niftyMidSmallcap400MomentumQuality100_list.csv",
    "Nifty Smallcap250 Momentum Quality 100": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250MomentumQuality100_list.csv",
    "Nifty Quality Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_quality_lowvol30list.csv",
    "Nifty50 Equal Weight": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "Nifty50 Value 20": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "Nifty500 Value 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Value50_list.csv",
    "Nifty200 Quality 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Quality30_list.csv",
    "Nifty Midcap 150": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150list.csv",
    "Nifty500 LargeMidSmall Equal-Cap Weighted":"https://www.niftyindices.com/IndexConstituent/ind_nifty500LargeMidSmallEqualCapWeighted_list.csv",
    "Nifty EV & New Age Automotive":"https://www.niftyindices.com/Index_Statistics/ind_niftyEv_NewAgeAutomotive_list.csv",
    "Nifty India Tourism":"https://www.niftyindices.com/IndexConstituent/ind_niftyindiatourism_list.csv",
    "Nifty Growth Sectors 15":"https://www.niftyindices.com/IndexConstituent/ind_NiftyGrowth_Sectors15_Index.csv",
    "Nifty500 Equal Weight":"https://www.niftyindices.com/IndexConstituent/ind_nifty500EqualWeight_list.csv",
    "NIFTY Alpha Quality Value Low-Volatility 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_value_lowvol30list.csv",
    "Nifty500 Momentum 50":"https://www.niftyindices.com/IndexConstituent/ind_nifty500Momentum50_list.csv",
    "Nifty200 Value 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty200Value30_list.csv",
    "Nifty Top 10 Equal Weight":"https://www.niftyindices.com/IndexConstituent/ind_niftytop10EqualWeight_list.csv"
}

# Trading name

In [ ]:
urls = { 
    "Nifty 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv",
    "Nifty Next 50": "https://www.niftyindices.com/IndexConstituent/ind_niftynext50list.csv",
    "Nifty 100": "https://www.niftyindices.com/IndexConstituent/ind_nifty100list.csv",
    "Nifty 200": "https://www.niftyindices.com/IndexConstituent/ind_nifty200list.csv",
    "NIFTY TOTAL MKT": "https://www.niftyindices.com/IndexConstituent/ind_niftytotalmarket_list.csv",
    "Nifty 500": "https://www.niftyindices.com/IndexConstituent/ind_nifty500list.csv",
    "NIFTY500 MULTICAP": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Multicap502525_list.csv",
    "Nifty Midcap150": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150list.csv",
    "Nifty Midcap 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap50list.csv",
    "NIFTY MID SELECT": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcapselect_list.csv",
    "NIFTY MIDCAP 100": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap100list.csv",
    "NIFTY SMLCAP 250": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap250list.csv",
    "NIFTY SMLCAP 50": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap50list.csv",
    "NIFTY SMLCAP 100": "https://www.niftyindices.com/IndexConstituent/ind_niftysmallcap100list.csv",
    "NIFTY MICROCAP250": "https://www.niftyindices.com/IndexConstituent/ind_niftymicrocap250_list.csv",
    "NIFTY LARGEMID250": "https://www.niftyindices.com/IndexConstituent/ind_niftylargemidcap250list.csv",
    "NIFTY MIDSML 400": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallcap400list.csv",
    "Nifty Auto": "https://www.niftyindices.com/IndexConstituent/ind_niftyautolist.csv",
    "Nifty Bank": "https://www.niftyindices.com/IndexConstituent/ind_niftybanklist.csv",
    "Nifty Fin Service": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancelist.csv",
    "NIFTY FINSRV25 50": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservices25-50list.csv",
    "Nifty Financial Services Ex Bank": "https://www.niftyindices.com/IndexConstituent/ind_niftyfinancialservicesexbank_list.csv",
    "Nifty FMCG": "https://www.niftyindices.com/IndexConstituent/ind_niftyfmcglist.csv",
    "NIFTY HEALTHCARE": "https://www.niftyindices.com/IndexConstituent/ind_niftyhealthcarelist.csv",
    "Nifty IT": "https://www.niftyindices.com/IndexConstituent/ind_niftyitlist.csv",
    "Nifty Media": "https://www.niftyindices.com/IndexConstituent/ind_niftymedialist.csv",
    "Nifty Metal": "https://www.niftyindices.com/IndexConstituent/ind_niftymetallist.csv",
    "Nifty Pharma": "https://www.niftyindices.com/IndexConstituent/ind_niftypharmalist.csv",
    "Nifty Pvt Bank": "https://www.niftyindices.com/IndexConstituent/ind_nifty_privatebanklist.csv",
    "Nifty PSU Bank": "https://www.niftyindices.com/IndexConstituent/ind_niftypsubanklist.csv",
    "Nifty Realty": "https://www.niftyindices.com/IndexConstituent/ind_niftyrealtylist.csv",
    "NIFTY CONSR DURBL": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumerdurableslist.csv",
    "NIFTY OIL AND GAS": "https://www.niftyindices.com/IndexConstituent/ind_niftyoilgaslist.csv",
    "Nifty MidSmall Financial Services": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallfinancailservice_list.csv",
    "Nifty MidSmall Healthcare": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallhealthcare_list.csv",
    "Nifty MidSmall IT & Telecom": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallitAndtelecom_list.csv",
    "Nifty India Corporate Group Index - Aditya Birla Group": "https://www.niftyindices.com/IndexConstituent/ind_nifty_adityabirlalist.csv",
    "Nifty Commodities": "https://www.niftyindices.com/IndexConstituent/ind_niftycommoditieslist.csv",
    "Nifty Core Housing": "https://www.niftyindices.com/IndexConstituent/ind_niftyCoreHousing_list.csv",
    "Nifty CPSE": "https://www.niftyindices.com/IndexConstituent/ind_niftycpselist.csv",
    "Nifty Energy": "https://www.niftyindices.com/IndexConstituent/ind_niftyenergylist.csv",
    "Nifty Housing": "https://www.niftyindices.com/IndexConstituent/ind_niftyhousing_list.csv",
    "Nifty Consumption": "https://www.niftyindices.com/IndexConstituent/ind_niftyconsumptionlist.csv",
    "Nifty India Defence": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadefence_list.csv",
    "NIFTY IND DIGITAL": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiadigital_list.csv",
    "NIFTY INDIA MFG": "https://www.niftyindices.com/IndexConstituent/ind_niftyindiamanufacturing_list.csv",
    "Nifty Infra": "https://www.niftyindices.com/IndexConstituent/ind_niftyinfralist.csv",
    "Nifty India Corporate Group Index - Mahindra Group": "https://www.niftyindices.com/IndexConstituent/ind_nifty_mahindralist.csv",
    "Nifty Mid Liq 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty_Midcap_Liquid15.csv",
    "Nifty MidSmall India Consumption": "https://www.niftyindices.com/IndexConstituent/ind_niftymidsmallindiaconsumption_list.csv",
    "Nifty MNC": "https://www.niftyindices.com/IndexConstituent/ind_niftymnclist.csv",
    "Nifty Mobility": "https://www.niftyindices.com/IndexConstituent/ind_niftymobility_list.csv",
    "Nifty PSE": "https://www.niftyindices.com/IndexConstituent/ind_niftypselist.csv",
    "Nifty REITs & InvITs": "https://www.niftyindices.com/IndexConstituent/ind_niftyREITs_InvITs_list.csv",
    "Nifty Non-Cyclical Consumer": "https://www.niftyindices.com/IndexConstituent/ind_niftynon-cyclicalconsumer_list.csv",
    "Nifty Serv Sector": "https://www.niftyindices.com/IndexConstituent/ind_niftyservicelist.csv",
    "Nifty India Corporate Group Index - Tata Group": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tatalist.csv",
    "Nifty India Corporate Group Index - Tata Group 25% Cap": "https://www.niftyindices.com/IndexConstituent/ind_nifty_tata25caplist.csv",
    "Nifty Transportation & Logistics": "https://www.niftyindices.com/IndexConstituent/ind_niftytransportationandlogistics%20_list.csv",
    "Nifty100 Liquid 15": "https://www.niftyindices.com/IndexConstituent/ind_Nifty100_Liquid15.csv",
    "Nifty500 Multicap India Manufacturing 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapIndiaManufacturing503020_list.csv",
    "Nifty500 Multicap Infrastructure 50:30:20": "https://www.niftyindices.com/IndexConstituent/ind_nifty500MulticapInfrastructure503020_list.csv",
    "NIFTY SME EMERGE": "https://www.niftyindices.com/IndexConstituent/ind_niftysmelist.csv",
    "NIFTY100 EQL WGT": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "NIFTY100 LOWVOL30": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "Nifty200Momentm30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Momentum30_list.csv",
    "Nifty200 Alpha 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200alpha30_list.csv",
    "Nifty100 Alpha 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Alpha30list.csv",
    "NIFTY ALPHA 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty_Alpha_Index.csv",
    "Nifty Alpha Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_lowvol30list.csv",
    "Nifty Alpha Quality Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_lowvol30list.csv",
    "Nifty Div Opps 50": "https://www.niftyindices.com/IndexConstituent/ind_niftydivopp50list.csv",
    "NIFTY HIGH BETA 50": "https://www.niftyindices.com/IndexConstituent/nifty_High_Beta50_Index.csv",
    "NIFTY LOW VOLATILITY 50": "https://www.niftyindices.com/IndexConstituent/nifty_low_Volatility50_Index.csv",
    "NIFTY100 QUALTY30": "https://www.niftyindices.com/IndexConstituent/ind_nifty100Quality30list.csv",
    "Nifty Midcap150 Momentum 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150momentum50_list.csv",
    "Nifty Midcap150 Quality 50": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150quality50list.csv",
    "Nifty Smallcap250 Quality 50 Index": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250_Quality50_list.csv",
    "Nifty MidSmallcap400 Momentum Quality 100": "https://www.niftyindices.com/IndexConstituent/ind_niftyMidSmallcap400MomentumQuality100_list.csv",
    "Nifty Smallcap250 Momentum Quality 100": "https://www.niftyindices.com/IndexConstituent/ind_niftySmallcap250MomentumQuality100_list.csv",
    "Nifty Quality Low Volatility 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty_quality_lowvol30list.csv",
    "NIFTY50 EQL WGT": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50EqualWeight.csv",
    "Nifty50 Value 20": "https://www.niftyindices.com/IndexConstituent/ind_Nifty50_Value20.csv",
    "Nifty500 Value 50": "https://www.niftyindices.com/IndexConstituent/ind_nifty500Value50_list.csv",
    "NIFTY200 QUALTY 30": "https://www.niftyindices.com/IndexConstituent/ind_nifty200Quality30_list.csv",
    "NIFTY MIDCAP 150": "https://www.niftyindices.com/IndexConstituent/ind_niftymidcap150list.csv",
    "Nifty500 LargeMidSmall Equal-Cap Weighted":"https://www.niftyindices.com/IndexConstituent/ind_nifty500LargeMidSmallEqualCapWeighted_list.csv",
    "Nifty EV & New Age Automotive":"https://www.niftyindices.com/Index_Statistics/ind_niftyEv_NewAgeAutomotive_list.csv",
    "Nifty India Tourism":"https://www.niftyindices.com/IndexConstituent/ind_niftyindiatourism_list.csv",
    "Nifty GrowSect 15":"https://www.niftyindices.com/IndexConstituent/ind_NiftyGrowth_Sectors15_Index.csv",
    "Nifty500 Equal Weight":"https://www.niftyindices.com/IndexConstituent/ind_nifty500EqualWeight_list.csv",
    "NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty_alpha_quality_value_lowvol30list.csv",
    "Nifty500 Momentum 50":"https://www.niftyindices.com/IndexConstituent/ind_nifty500Momentum50_list.csv",
    "Nifty200 Value 30":"https://www.niftyindices.com/IndexConstituent/ind_nifty200Value30_list.csv",
    "Nifty Top 10 Equal Weight":"https://www.niftyindices.com/IndexConstituent/ind_niftytop10EqualWeight_list.csv"
}